In [1]:
import sys
sys.path.append('/Users/Ziqi/Desktop/developer/mgwr/')
from mgwr.gwr import GWR
from mgwr.FastGWR import FastGWR
from mgwr.sel_bw import Sel_BW
import pandas as pd
import numpy as np

In [2]:
zillow = pd.read_csv("/Users/Ziqi/Desktop/developer/gwr-hpc/zillow_10k.csv",sep=',')
zillow.head()
y = zillow.value.as_matrix().reshape(-1,1)
X = zillow.iloc[:,3:].as_matrix()
k = zillow.shape[1]
u = zillow.utmX
v = zillow.utmY
n = zillow.shape[0]
coords = list(zip(u,v))

In [ ]:
#Adaptive Bisquare

In [ ]:
%%time 
fast_rslt = FastGWR(coords,y,X).fit(nproc=2) #FastGWR fitting with optimal bw seaching

In [ ]:
%%time 
bw = Sel_BW(coords,y,X).search() #MGWR fitting with optimal bw seaching
rslt = GWR(coords,y,X,bw=bw).fit()

In [ ]:
#tr_S
print(rslt.tr_S)
print(fast_rslt.tr_S)
#AICc
print(rslt.aicc)
print(fast_rslt.aicc)
#Sigma2
print(rslt.sigma2)
print(fast_rslt.sigma2)
#Estimtaes
print(np.allclose(fast_rslt.params, rslt.params))
print(np.allclose(fast_rslt.bse, rslt.bse))
print(np.allclose(fast_rslt.predy, rslt.predy.reshape(-1)))

In [ ]:
#Fixed Gaussian

In [ ]:
%%time
fast_rslt = FastGWR(coords,y,X,fixed=True).fit(nproc=2) #FastGWR fitting with optimal bw seaching

In [ ]:
%%time
bw = Sel_BW(coords,y,X,fixed=True,kernel='gaussian').search() #MGWR fitting with optimal bw seaching
print(bw)
rslt = GWR(coords,y,X,bw=bw,fixed=True,kernel='gaussian').fit()

In [ ]:
#tr_S
print(rslt.tr_S)
print(fast_rslt.tr_S)
#AICc
print(rslt.aicc)
print(fast_rslt.aicc)
#Sigma2
print(rslt.sigma2)
print(fast_rslt.sigma2)
#Estimtaes
print(np.allclose(fast_rslt.params, rslt.params))
print(np.allclose(fast_rslt.bse, rslt.bse))
print(np.allclose(fast_rslt.predy, rslt.predy.reshape(-1)))